In [3]:
# -*- coding: utf-8 -*-

#声明Notebook类型，必须在引入pyecharts.charts等模块前声明
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

from pyecharts import options as opts
from pyecharts.charts import Scatter,Bar,Line
from impala.dbapi import connect

#连接Hadoop数据库
v1 = []
v2 = []
v3 = []
v4 = []
conn = connect(host='192.168.1.7', port=10000, database='sales',auth_mechanism='NOSASL',user='root')
cursor = conn.cursor()

#读取Hadoop表数据
sql_num = "SELECT region,ROUND(SUM(sales)/10000,2),ROUND(SUM(profit)/10000,2),ROUND(SUM(amount),2) FROM customers,orders WHERE customers.cust_id=orders.cust_id and dt=2019 GROUP BY region"
cursor.execute(sql_num)
sh = cursor.fetchall()
for s in sh:
    v1.append(s[0])
    v2.append(s[1])
    v3.append(s[2])
    v4.append(s[3]) 

#柱形图与折线图组合
def overlap_bar_line() -> Bar:
    bar = (
        Bar()
        .add_xaxis(v1)
        .add_yaxis("销售额", v2)
        .add_yaxis("利润额", v3)
        .extend_axis(
            yaxis=opts.AxisOpts(
                axislabel_opts=opts.LabelOpts(formatter="{value} 件"), interval=500
            )
        )
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="区域销售业绩比较分析", subtitle="2019年企业经营状况分析"),
            toolbox_opts=opts.ToolboxOpts(),
            yaxis_opts=opts.AxisOpts(
                axislabel_opts=opts.LabelOpts(formatter="{value} 万元"), interval=20
            ),
        )
    )
    
    line = Line().add_xaxis(v1).add_yaxis("销售数量", v4, yaxis_index=1)             
    bar.overlap(line)
    return bar
    
#第一次渲染时候调用load_javasrcript文件
overlap_bar_line().load_javascript()
#展示数据可视化图表
overlap_bar_line().render_notebook()